In [1]:
# ruff: noqa
import argparse
import logging
import os
import sys
import warnings
from os.path import join

from spatialdata import read_zarr

sys.path.insert(1, "/dss/dsshome1/0C/ra98gaq/Git/cellseg-benchmark")

from cellseg_benchmark import sdata_utils as su

/home/ubuntu/miniforge3/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/ubuntu/miniforge3/lib/python3.12/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)


In [2]:
#warnings.filterwarnings("ignore")

In [3]:
logger = logging.getLogger("shape_mapping")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s]: %(message)s"))
logger.addHandler(handler)

In [12]:
def get_args(test_args=None):  # noqa: D103
    p = argparse.ArgumentParser(
    description="Creates a master sdata for a given sample, containing multiple segmentation results."
    )
    p.add_argument("sample", help="Sample name.")
    p.add_argument(
        "data_path",
        help="Path to folder with merscope output data (e.g. /cohort1/slide2/region0).",
    )
    p.add_argument(
        "zmode", choices=["z3"], help="Mode of master sdata. Either 'z3' or '3d' (currently only z3 is implemented)."
    )
    p.add_argument("data_dir", help="Output data folder.")
    p.add_argument(
        "--n_ficture",
        default=21,
        type=int,
        help="Consider Ficture model with n_ficture factors.",
    )
    p.add_argument("--run_date", type=str, help="run date (YYYYMMDD).", default=None)
    p.add_argument("--organism", type=str, help="organism.", default=None)
    p.add_argument("--slide", type=str, help="slide.", default=None)
    p.add_argument("--region", type=str, help="region.", default=None)
    p.add_argument("--cohort", type=str, help="cohort.", default=None)
    p.add_argument("--obs", action="append", default=[], metavar="KEY=VAL",
        help="Extra covariates to add to adata.obs (repeatable), e.g. --obs tissue=brain.")
    if test_args is not None:
        return p.parse_args(test_args)
    else:
        return p.parse_args()

In [13]:
# Simulate CLI arguments inside Jupyter
args = get_args(
    [
        "SynergyLung_s1_r0",
        "/dss/dssfs03/pn52re/pn52re-dss-0000/Synergy-projects-temp/MouseLung-Sijia/output-20250704_AGLiesz-Sijia-MouseLung-Slide1-Stroke_VMSC03901/region_R1-Stroke",
        "z3",
        "/dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark",
        "--cohort",
        "SynergyLung",
        "--slide",
        "1",
        "--region",
        "0",
        "--organism",
        "mouse",
        "--run_date",
        "20250704",
        "--obs",
        "condition=stroke",
    ]
)

In [14]:
args

Namespace(sample='SynergyLung_s1_r0', data_path='/dss/dssfs03/pn52re/pn52re-dss-0000/Synergy-projects-temp/MouseLung-Sijia/output-20250704_AGLiesz-Sijia-MouseLung-Slide1-Stroke_VMSC03901/region_R1-Stroke', zmode='z3', data_dir='/dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark', n_ficture=21, run_date='20250704', organism='mouse', slide='1', region='0', cohort='SynergyLung', obs=['condition=stroke'])

In [15]:
NONES = {"", "None", "none", "null", "NULL", None}
for k in ["organism", "slide", "region", "cohort"]:
    if getattr(args, k) in NONES:
        setattr(args, k, None)

In [17]:
extra_obs = {}
for kv in args.obs:
    k, v = kv.split("=", 1)
    extra_obs[k] = None if v in NONES else v

In [25]:
extra_obs

{'condition': 'stroke'}

In [20]:
logger.info("Importing images and points...")
su.process_merscope(args.sample, args.data_dir, args.data_path, zmode=args.zmode)

2025-10-20 19:04:37,327 [INFO]: Importing images and points...


INFO     The column "global_x" has now been renamed to "x"; the column "x" was already present in the dataframe,   
         and will be dropped.                                                                                      
INFO     The column "global_y" has now been renamed to "y"; the column "y" was already present in the dataframe,   
         and will be dropped.                                                                                      


/home/ubuntu/miniforge3/lib/python3.12/functools.py:946: UserWarning: The index of the dataframe is not monotonic increasing. It is recommended to sort the data to adjust the order of the index before calling .parse() (or call `parse(sort=True)`) to avoid possible problems due to unknown divisions.
  return method.__get__(obj, cls)(*args, **kwargs)


INFO     The Zarr backing store has been changed from None the new file path:                                      
         /dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark/samples/SynergyLung_s1_r0/sdata_z3.zarr             


version mismatch: detected: RasterFormatV02, requested: FormatV04
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compre

In [21]:
sdata_path = join(args.data_dir, "samples", args.sample)
sdata_main = read_zarr(join(sdata_path, "sdata_z3.zarr"))

version mismatch: detected: RasterFormatV02, requested: FormatV04
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compre

In [22]:
logger.info("Integrating segmentation data from available methods into main sdata...")
# only work on methods with actual data available
seg_methods = [
    method
    for method in os.listdir(join(sdata_path, "results"))
    if os.path.isdir(join(sdata_path, "results", method, "sdata.zarr"))
]

2025-10-20 19:14:42,498 [INFO]: Integrating segmentation data from available methods into main sdata...


In [23]:
seg_methods

['Cellpose_1_Merlin']

In [24]:
su.integrate_segmentation_data(
    sdata_path,
    seg_methods,
    sdata_main,
    run_date=args.run_date,
    organism=args.organism,
    slide=args.slide,
    region=args.region,
    cohort=args.cohort,
    write_to_disk=True,
    data_path=args.data_path,
    logger=logger,
    **extra_obs,
)

  0%|          | 0/1 [00:00<?, ?it/s]2025-10-20 19:14:42,591 [INFO]: Adding Cellpose_1_Merlin...
2025-10-20 19:14:50,133 [INFO]: Adding shapes of Cellpose_1_Merlin...
2025-10-20 19:14:50,850 [INFO]: Adding adata for Cellpose_1_Merlin...
/home/ubuntu/miniforge3/lib/python3.12/site-packages/spatialdata/_core/_elements.py:125: UserWarning: Key `adata_Cellpose_1_Merlin` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)
2025-10-20 19:14:51,174 [WARNING]: No annotation files found for Cellpose_1_Merlin. Skipping
2025-10-20 19:14:54,243 [INFO]: calculate volume metrics Cellpose_1_Merlin
/home/ubuntu/miniforge3/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/

SpatialData object, with associated Zarr store: /dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark/samples/SynergyLung_s1_r0/sdata_z3.zarr
├── Images
│     └── 'SynergyLung_s1_r0_z3': DataTree[cyx] (7, 72868, 117555), (7, 36434, 58777), (7, 18217, 29388), (7, 9108, 14694), (7, 4554, 7347)
├── Points
│     └── 'SynergyLung_s1_r0_transcripts': DataFrame with shape: (<Delayed>, 10) (2D points)
├── Shapes
│     └── 'boundaries_Cellpose_1_Merlin': GeoDataFrame shape: (113375, 9) (2D shapes)
└── Tables
      └── 'adata_Cellpose_1_Merlin': AnnData (113375, 451)
with coordinate systems:
    ▸ 'global', with elements:
        SynergyLung_s1_r0_z3 (Images), SynergyLung_s1_r0_transcripts (Points), boundaries_Cellpose_1_Merlin (Shapes)
    ▸ 'micron', with elements:
        SynergyLung_s1_r0_z3 (Images), SynergyLung_s1_r0_transcripts (Points), boundaries_Cellpose_1_Merlin (Shapes)
    ▸ 'pixel', with elements:
        SynergyLung_s1_r0_z3 (Images), SynergyLung_s1_r0_transcripts (Points), bounda

In [ ]:
#################

In [ ]:
# test sdata

In [37]:
sdata_main.tables["adata_Cellpose_1_Merlin"].obs.columns

Index(['fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x',
       'max_y', 'anisotropy', 'transcript_count', 'perimeter_area_ratio',
       'Txnip_raw', 'Txnip_high_pass', 'Fth1_raw', 'Fth1_high_pass',
       'DAPI_raw', 'DAPI_high_pass', 'Scgb1a1_raw', 'Scgb1a1_high_pass',
       'Sftpc_raw', 'Sftpc_high_pass', 'PolyT_raw', 'PolyT_high_pass',
       'Ifitm3_raw', 'Ifitm3_high_pass', 'region', 'slide', 'dataset_id',
       'cells_region', 'cell_id', 'spt_region', 'area', 'volume_sum',
       'volume_final', 'num_z_planes', 'size_normalized',
       'surface_to_volume_ratio', 'sphericity', 'solidity', 'elongation',
       'condition', 'run_date', 'organism', 'cohort', 'sample'],
      dtype='object')

In [40]:
sdata_main.tables["adata_Cellpose_1_Merlin"].obs

,fov,volume,center_x,center_y,min_x,min_y,max_x,max_y,anisotropy,transcript_count,...,size_normalized,surface_to_volume_ratio,sphericity,solidity,elongation,condition,run_date,organism,cohort,sample
EntityID,,,,,,,,,,,,,,,,,,,,,
2871360300009100179,393,2279.483333,4827.389643,298.186576,4819.689472,290.181249,4835.230607,306.499960,1.071683,3,...,13.782487,0.261707,0.965885,0.993433,0.020009,stroke,20250704,mouse,SynergyLung,SynergyLung_s1_r0
2871360300009100433,394,1961.525662,4769.783387,491.299580,4762.258860,484.373616,4776.750034,499.173832,1.106673,3,...,12.785166,0.284560,0.949398,0.996844,0.124480,stroke,20250704,mouse,SynergyLung,SynergyLung_s1_r0
2871360300010100013,533,786.300528,5623.826510,3.984424,5617.563526,1.086482,5629.996663,7.775898,1.858628,0,...,8.094754,0.483947,0.818856,0.996692,0.037615,stroke,20250704,mouse,SynergyLung,SynergyLung_s1_r0
2871360300010100021,533,1365.077774,5636.820215,12.129918,5631.200488,5.256492,5642.705877,18.575755,1.164090,0,...,10.665668,0.342483,0.941793,0.993160,0.022361,stroke,20250704,mouse,SynergyLung,SynergyLung_s1_r0
2871360300010100032,533,1430.555153,5652.493755,18.983835,5645.132427,13.721422,5659.371952,24.763578,1.291070,0,...,10.918467,0.338499,0.919965,0.990455,0.132803,stroke,20250704,mouse,SynergyLung,SynergyLung_s1_r0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2871360300386100266,646,2943.737960,5937.240619,7587.430785,5928.289015,7577.942621,5945.583826,7596.829379,1.145079,3,...,15.662423,0.231594,0.955078,0.993341,0.237049,stroke,20250704,mouse,SynergyLung,SynergyLung_s1_r0
2871360300386100288,649,2810.073233,6087.837774,7618.854011,6079.130175,7609.490919,6096.435894,7628.708010,1.016250,3,...,15.302705,0.239064,0.938958,0.984662,0.102386,stroke,20250704,mouse,SynergyLung,SynergyLung_s1_r0
2871360300387100060,723,2774.095048,6295.099854,7355.708631,6286.646981,7346.610796,6303.857768,7364.136669,1.090561,3,...,15.204427,0.236305,0.973473,0.996720,0.062222,stroke,20250704,mouse,SynergyLung,SynergyLung_s1_r0
